## Final Project Submission

You'll clean, explore, and model this dataset with a multivariate linear regression to predict the sale price of houses as accurately as possible.

Please fill out:
* Student name: Caitlin Snyder
* Student pace: self-paced
* Scheduled project review date/time: 
* Instructor name: Jeff Herman
* Blog post URL: 



# 1) Introduction

As I wrap up Module 3 of Flat Iron's Data Science bootcamp, I will be 

...

Follow along below, or take a look at the [Jupyter notebook]().

https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/page/25/


I'll try to answer these questions below--let's get started!

# 2) Import the data 

Import the relevant libraries:

In [291]:

# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = 'all'


import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import datetime as dt
pd.options.mode.chained_assignment = None

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import stats


from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

class Paths:
    def __init__(self):
        self.train_values = 'data/training_set_values.csv'
        self.train_labels = 'data/training_set_labels.csv'
        self.test_values = 'data/test_set_values.csv'


Read in and preview the data:

In [2]:
class DataKeeper:
    def __init__(self):
        self.outcome = 'status_group'
        paths = Paths()
        
        X_train = self.load_data(paths.train_values)
        y_train = self.load_data(paths.train_labels)
        
        self.X_test = self.load_data(paths.test_values)
        self.train = pd.concat([X_train, y_train], axis=1, join="inner")
        
    
    def load_data(self, path):
        df = pd.read_csv(path)
        df.set_index('id', inplace=True)
        print(path, df.shape)
        return df

dfs = DataKeeper()

data/training_set_values.csv (59400, 39)
data/training_set_labels.csv (59400, 1)
data/test_set_values.csv (14850, 39)


Examining the training data, we see that we have 59,400 rows and 40 columns. We also call describe() on the dataframe to get an overview of the descriptive statistics corresponding to each attribute:

In [ ]:
dfs.train.describe()

Preview the relationships:

In [ ]:
# pd.plotting.scatter_matrix(dfs.train, figsize=(10,10)); 

# 3) Prepare the data

Reviewing the data types, we see that there are a number of string columns. We need to examine each of these to see what kind of cleaning may be necessary.

In [ ]:
dfs.train.dtypes

# id                         int64
# amount_tsh               float64
# date_recorded             object
# funder                    object
# gps_height                 int64
# installer                 object
# longitude                float64
# latitude                 float64
# wpt_name                  object
# num_private                int64
# basin                     object
# subvillage                object
# region                    object
# region_code                int64
# district_code              int64
# lga                       object
# ward                      object
# population                 int64
# public_meeting            object
# recorded_by               object
# scheme_management         object
# scheme_name               object
# permit                    object
# construction_year          int64
# extraction_type           object
# extraction_type_group     object
# extraction_type_class     object
# management                object
# management_group          object
# payment                   object
# payment_type              object
# water_quality             object
# quality_group             object
# quantity                  object
# quantity_group            object
# source                    object
# source_type               object
# source_class              object
# waterpoint_type           object
# waterpoint_type_group     object
# dtype: object

When we preview the unique values for each column, we can see that there are common but differently formatted values in the 'funder' and 'installer' columns. For example, 'Brown' appears in column 'funder' where as 'brown' appears in column 'installer.' To align these columns, we'll lowercase both.

In [ ]:
def get_str_cols(df):
    return df.select_dtypes(include=['object']).columns

def show_unique_values(df, cols):
    exclude = ['date_recorded']
    for col in cols:
        if col not in exclude:
            print(col)
            print(df[col].unique())
        
def cols_to_lower(df, cols):
    for col in cols:
        df[col] = df[col].str.lower()
        
str_cols = get_str_cols(dfs.train)
show_unique_values(dfs.train, str_cols)
cols_to_lower(dfs.train, ['funder', 'installer'])

# funder
# ['roman' 'grumeti' 'lottery club' ... 'dina' 'brown' 'samlo']
# installer
# ['roman' 'grumeti' 'world vision' ... 'dina' 'brown' 'selepta']


We also need to type 'date_recorded' as numeric in order for our ML classifiers to accept the values:

In [ ]:
def convert_to_date(df, cols):
    for col in cols:
        df[col] = pd.DatetimeIndex(pd.to_datetime(df[col]))
        df[col]=df[col].map(dt.datetime.toordinal)
        
convert_to_date(dfs.train, ['date_recorded'])


The remaining string columns look correctly typed. We'll one-hot encode these values below.

### Identify null values

We see that we have a fair number of null values. For the most part, the percentage of null values is relatively low (< 7%). However, the number of nulls in 'scheme_name' is pretty massive: 47.4%! With nulls making up nearly half of all values in the column, we're better off dropping this feature all together.

In [ ]:
def get_na_cols(df):
    na_sum = df.isna().sum()
    na_cols = na_sum[na_sum != 0]
    total = df.shape[0]
    if len(na_cols) == 0:
        print('No nans')
    for i in na_cols.index:
        cnt = na_cols[i]
        pcnt = round((cnt*100)/total, 1)
        print(f"{i}: {cnt} ({pcnt}%)")

        
def drop_cols(df, cols):
    for col in cols:
        if col in df.columns:
            df.drop([col], axis=1, inplace=True)
        
get_na_cols(dfs.train)

# funder: 3635 (6.1%)
# installer: 3655 (6.2%)
# subvillage: 371 (0.6%)
# public_meeting: 3334 (5.6%)
# scheme_management: 3877 (6.5%)
# scheme_name: 28166 (47.4%)
# permit: 3056 (5.1%)

drop_cols(dfs.train, ['scheme_name'])

We can see that the dataset also contains "unknown" values. Let's consolidate the nans and unknown values:

In [ ]:
def make_unknowns_nan(df):
    df.replace('unknown', np.nan, inplace=True)
    
make_unknowns_nan(dfs.train)


### One-hot encode categorical features

In [ ]:
def get_ohe(df):
    ohe = OneHotEncoder()
    ohe.fit(df)
    return ohe.transform(df).toarray()

ohe_train = get_ohe(dfs.train)


Preview the mean and standard deviation associated with outcome type (functionality):

In [ ]:
aggs = dfs.train.groupby(dfs.train[dfs.outcome]).agg(['mean', 'std'])
aggs

### Split the data

We can first define a class to split our data into training and testing subsets and peform the necessary transformations on our continuous and categorical features.

### Consolidate our data cleaning steps

We can update our DataKeeper class and add additional helper classes to perform all of the above data cleaning steps:

In [292]:
class ContinuousHelper():
    def __init__(self):
        pass
    
    def get_cleaned_df(self, df, features):
        self.impute_zeros(df, ['construction_year', 'population'])
        list(map(lambda col: self.handle_zeros_pre_log(df, col), features))
        list(map(lambda col: self.handle_negatives_pre_log(df, col), features))
        
    def impute_zeros(self, df, cols):
        for col in cols:
            df[col] = df.apply(
                lambda row: np.nan if row[col] == 0 else row[col], axis=1)
            self.impute_median(df, col)
    
    def impute_median(self, df, col):
        df_one_col = df[[col]]
        df_one_col.fillna(df_one_col.median(), inplace=True)        
        df[col] = df_one_col[col]
            
    def handle_zeros_pre_log(self, df, col):
        if df[col].min() != 0:
            return
        elif (df[col].min() == 0 and df[col].max() == 0):
            df.drop([col],axis=1, inplace=True)
        else:
            col_non_zero_min = df.loc[df[col] > 0, col].min()
            offset = col_non_zero_min/2
            df[col] = df.apply(
                lambda row: row[col] + offset,
                axis=1)
            
    def handle_negatives_pre_log(self, df, col):
        if df[col].min() >= 0:
            return
        else:
            col_min = abs(df[col].min()) + 1
            df[col] = df.apply(
                lambda row: row[col] + col_min,
                axis=1)


In [304]:
class FuzzyMatcher:
    def __init__(self):
        self.cols = []
    
    def clean_orgs(self, df, cols):
        self.cols = cols
        self.clean_col_text(df)
        
        scores_df = self.get_matches(df)       
        self.replace_with_matches(df, scores_df) 
        
    def clean_col_text(self, df):
        for col in self.cols:
            df[col] = df[col].str.lower()
            df[col] = df.apply(lambda row: self.manually_clean_col_text(str(row[col])), axis=1)
            df[col] = df.apply(lambda row: self.remove_special_chars(str(row[col])), axis=1)
            
            
    def manually_clean_col_text(self, value):
        replace_dict = { # ideally in constants file
            "private individual": "private",
            "not known": "unknown",
            "0": "unknown",
            "-": "unknown",
            "nan": "unknown",
            "action in a": "action in africa",
            "wateraid": "water aid"
        }
        if value in replace_dict.keys():
            value = replace_dict[value]
        return value
    
    def remove_special_chars(self, value):
        special_chars = [".", "/", "-", "[", "]", "(", ")"]
        for char in special_chars:
            value = value.replace(char, "")
        return value
    
    def get_matches(self, df):
        values = self.get_orgs(df)
        match_df = self.get_match_df(values)
        scores_df = self.get_scores(match_df)
        return scores_df

    def get_orgs(self, df):
        list_of_lists = [df[col] for col in self.cols]
        orgs = [j for sub in list_of_lists for j in sub]
        return np.unique([str(i).lower() for i in orgs if str(i) != 'nan']).tolist()
    
    def get_match_df(self, values):
        score_sort = [(x,) + i
                     for x in values 
                     for i in process.extract(x, values, scorer=fuzz.token_sort_ratio)]
        match_df = pd.DataFrame(score_sort, columns=['name_sort','match_sort','score_sort'])
        match_df['sorted_name_sort'] = \
            np.minimum(match_df['name_sort'], match_df['match_sort'])
        return match_df
    
    def get_scores(self, match_df):
        high_score_sort = self.get_score_sort(match_df)
        scores = self.get_score_groups(high_score_sort)
        scores_df = self.get_score_frame(scores)
        return scores_df

    def get_score_sort(self, match_df):
        high_score_sort = \
            match_df.loc[(match_df['score_sort'] >= 80) &
                    (match_df['name_sort'] !=  match_df['match_sort']) &
                    (match_df['sorted_name_sort'] != match_df['match_sort'])]
        
        high_score_sort = high_score_sort.drop('sorted_name_sort', axis=1).copy()
        return high_score_sort
        

    def get_score_groups(self, high_score_sort):
        return high_score_sort.groupby(['name_sort','score_sort']).agg(
                        {'match_sort': ', '.join}).sort_values(
                        ['score_sort'], ascending=False)
        
    def get_score_frame(self, scores):
        frame = { 
            'name_sort': scores.index.get_level_values(0), 
            'score_sort': scores.index.get_level_values(1),
            'match_sort': list(map(lambda x: x[0], scores.values.tolist())),
        }
        return pd.DataFrame(frame)
    
    def replace_with_matches(self, df, df_matches):
        for col in self.cols:
            for name in df_matches["name_sort"]:
                if name in df[col].values.tolist():
                    replace_value = df_matches.loc[df_matches["name_sort"] == name,
                                                   "match_sort"].values.tolist()[0]
                    df[col].replace(name, replace_value, inplace=True)



In [300]:
class CategoricalHelper():
    def __init__(self):
        self.fuzzy_matcher = FuzzyMatcher()
    
    def get_cleaned_df(self, df, features):
        df.fillna('unknown', inplace=True) 
        
    def match_names(self, df, cols):
        for col in cols:
            pass
            
            

In [301]:
class Cleaner():
    def __init__(self):
        self.cont_helper = ContinuousHelper()
        self.cat_helper = CategoricalHelper()
        
    def get_cleaned_df(self, df):
        self.general_cleaning(df)
        self.cat_helper.get_cleaned_df(df, self.get_cat_features(df))
        self.cont_helper.get_cleaned_df(df, self.get_cont_features(df))
        return df
    
    def general_cleaning(self, df):
        self.convert_to_date(df, ['date_recorded'])
        self.drop_cols(df, ['scheme_name', 'date_recorded']) # why dropping date recorded?
        
    def convert_to_date(self, df, cols):
        for col in cols:
            df[col] = pd.DatetimeIndex(pd.to_datetime(df[col]))
            df[col] = df[col].map(dt.datetime.toordinal)
    
    def drop_cols(self, df, cols):
        for col in cols:
            if col in df.columns:
                df.drop([col], axis=1, inplace=True)
                
    def get_cat_features(self, df):
        self.change_type(df, 'region_code', str)
        self.change_type(df, 'district_code', str)
        self.change_type(df, 'num_private', str)
        return df.select_dtypes(include=['object']).columns
    
    def change_type(self, df, col, new_type):
        df[col] = df[col].astype(new_type)
        
    def get_cont_features(self, df):
        return df.select_dtypes(exclude=['object']).columns
    
        

In [302]:
class Splitter:
    def __init__(self):
        self.split_dfs = {'X_train': None, 'X_test': None, 'y_train': None, 'y_test': None}
        
    def get_splits(self, df, outcome):
        self.set_splits(df, outcome)
        return self.split_dfs
    
    def set_splits(self, df, outcome):
        preds = [i for i in df.columns if i != outcome]
        X = df[preds]
        y = df[[outcome]]
        
        self.split_dfs['X_train'], self.split_dfs['X_test'], self.split_dfs['y_train'], self.split_dfs['y_test'] =\
            train_test_split(X, y, test_size = 0.2, random_state = 42)
        
        list(map(lambda key: self.apply_transformations(key), ['X_train', 'X_test']))
        self.match_columns()
            
            
    def apply_transformations(self, key):
        print(key, ': in')
        
        df = self.split_dfs[key]
        features = self.get_feature_types(df)
        
        df_cat = self.transform_cat(df[features['cat']])
        df_cont = self.transform_cont(df[features['cont']])
                
        merged_df = pd.concat([df_cat, df_cont], axis=1)
        self.split_dfs[key] = merged_df
        
        print(key, ': out')

    def get_feature_types(self, df):
        cat_features = df.select_dtypes(include=['object']).columns
        cont_features = df.select_dtypes(exclude=['object']).columns
        return {'cat': cat_features, 'cont': cont_features}
        
    def transform_cont(self, df):
        df_log = np.log(df)
        df_log.columns = [f'{column}_log' for column in df.columns]
        return df_log.apply(self.normalize)
    
    def normalize(self, feature):
        return (feature - feature.mean()) / feature.std()

    def transform_cat(self, df_cat):
        df_cat = df_cat.astype(str)
        id_index = df_cat.index
        ohe = OneHotEncoder(handle_unknown='ignore')
        ohe.fit(df_cat)
        
        matrix_ohe = ohe.transform(df_cat).toarray()
        df_ohe = pd.DataFrame(matrix_ohe, columns=ohe.get_feature_names(df_cat.columns))
        df_ohe['id'] = id_index
        df_ohe.set_index('id', drop=True, inplace=True)
        return df_ohe
    
    def match_columns(self):
        train_cols = self.split_dfs['X_train'].columns.values.tolist()
        test_cols = self.split_dfs['X_test'].columns.values.tolist()
        print(0)
        self.add_match_cols(train_cols, test_cols, 'X_test')
        print(1)
        self.add_match_cols(test_cols, train_cols, 'X_train')
        print(2)
        
    def add_match_cols(self, source_cols, target_cols, target_df_key):
        for col in source_cols:
            if col not in target_cols:
                print(col)
                self.split_dfs[target_df_key][col] = 0
        



In [303]:
class ModelReport:
    def __init__(self):
        self.splits = None
        self.accuracy_report = {'tree': 0, 'svm': 0, 'knn': 0}
    
    def get_reports(self, splits):
        self.splits = splits
        self.get_decision_tree_report()
        self.get_svm_report()
        self.get_knn_report()
        self.display_results()
        
    def get_decision_tree_report(self):
        clf = DecisionTreeClassifier(criterion='entropy')
        clf.fit(self.splits['X_train'], self.splits['y_train'])
        y_pred_tree = clf.predict(self.splits['X_test'])
        self.accuracy_report['tree'] = accuracy_score(self.splits['y_test'], y_pred_tree)

    def show_decision_tree_plot(self, outcome_values): # Not actively used
        features = self.get_features()
        fig, axes = plt.subplots(nrows = 1,ncols = 1, figsize = (3,3), dpi=300)
        tree.plot_tree(clf,
                       feature_names=features, 
                       class_names=outcome_values.astype('str'),
                       filled = True)
        plt.show()
        
    def get_features(self):
        self.features = self.splits['X_train'].columns.values.tolist() + \
            self.splits['y_train'].columns.values.tolist()
        
    def get_svm_report(self):
        SVC_model = SVC()
        SVC_model.fit(self.splits['X_train'], self.splits['y_train'])
        SVC_prediction = SVC_model.predict(self.splits['X_test'])
        self.accuracy_report['svm'] = accuracy_score(SVC_prediction, self.splits['y_test'])
    
    def get_knn_report(self):
        KNN_model = KNeighborsClassifier(n_neighbors=5)
        KNN_model.fit(self.splits['X_train'], self.splits['y_train'])
        KNN_prediction = KNN_model.predict(self.splits['X_test'])
#         print(confusion_matrix(SVC_prediction, dfs.splits['y_test']))
#         print(classification_report(KNN_prediction, dfs.splits['y_test']))
        self.accuracy_report['knn'] = accuracy_score(KNN_prediction, self.splits['y_test'])

    def display_results(self):
        print('---------\nAccuracy reports\n---------')
        for key in self.accuracy_report.keys():
            print(key, self.accuracy_report[key])
    

In [305]:
class DataKeeper:
    def __init__(self, run_type_dev=False):
        self.run_type_dev = run_type_dev
        self.outcome = 'status_group'
        self.outcome_values = []
        
        self.cleaned_df = None
        self.splits = None
        
        self.paths = Paths()
        self.cleaner = Cleaner()
        self.splitter = Splitter()
        self.report = ModelReport()
        
        self.get_data()

    def get_data(self):
        df = self.load_df()
        
        self.fuzzy_matcher.clean_orgs(df, ['funder', 'installer'])
        self.preprocess(df)
        
    def load_df(self):
        X_train = self.load_data(self.paths.train_values)
        y_train = self.load_data(self.paths.train_labels)
        
        if self.run_type_dev:
            X_train = X_train.iloc[0:50]
            y_train = y_train.iloc[0:50]
            
        df = pd.concat([X_train, y_train], axis=1, join="inner")
        self.outcome_values = np.unique(df[self.outcome])
        return df
    
    def load_data(self, path):
        df = pd.read_csv(path)
        df.set_index('id', inplace=True)
        return df
        
    def preprocess(self, raw_df):
        self.cleaned_df = self.cleaner.get_cleaned_df(raw_df)
        self.splits = self.splitter.get_splits(self.cleaned_df, self.outcome)
        
    def get_report(self):
        self.report.get_reports(self.splits)
    
    

In [306]:
dfs = DataKeeper(run_type_dev=False)

X_train : in
X_train : out
X_test : in
X_test : out
0
1
2


In [ ]:
dfs.get_report()


# 8) Conclusion

...

### Sources



In [ ]:
https://towardsdatascience.com/fuzzywuzzy-find-similar-strings-within-one-column-in-a-pandas-data-frame-99f6c2a0c212
    
https://towardsdatascience.com/fuzzywuzzy-fuzzy-string-matching-in-python-beginners-guide-9adc0edf4b35

https://stackabuse.com/overview-of-classification-methods-in-python-with-scikit-learn/
    
https://stackabuse.com/the-naive-bayes-algorithm-in-python-with-scikit-learn/